In [1]:
# ==========================================
# PHƯƠNG PHÁP 1: ENHANCED DATA AUGMENTATION
# ==========================================

# CELL 1: SETUP & GPU CHECK
!nvidia-smi
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("✅ GPU ENABLED!")
else:
    print("❌ NO GPU! Please enable GPU in Runtime settings.")

Thu Feb  5 13:03:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# CELL 2: IMPORT LIBRARIES
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

In [3]:
# CELL 3: MOUNT DRIVE & CONFIG PATHS
from google.colab import drive
drive.mount('/content/drive')

# Cấu hình đường dẫn
BASE_DIR = '/content/drive/MyDrive/CaptoneProject'
ZIP_PATH = f'{BASE_DIR}/camera.zip'
LOCAL_PATH = '/content/dataset'
CHECKPOINT_DIR = f'{BASE_DIR}/checkpoints/method1_aug'

# Tạo thư mục lưu checkpoint
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
MODEL_CHECKPOINT_PATH = f'{CHECKPOINT_DIR}/model_checkpoint.keras'
BEST_MODEL_PATH = f'{CHECKPOINT_DIR}/best_model.keras'
HISTORY_PATH = f'{CHECKPOINT_DIR}/training_history.pkl'

print(f"📂 Dataset ZIP: {ZIP_PATH}")
print(f"💾 Checkpoints will be saved to: {CHECKPOINT_DIR}")

Mounted at /content/drive
📂 Dataset ZIP: /content/drive/MyDrive/CaptoneProject/camera.zip
💾 Checkpoints will be saved to: /content/drive/MyDrive/CaptoneProject/checkpoints/method1_aug


In [4]:
# CELL 4: EXTRACT DATASET
if not os.path.exists(LOCAL_PATH):
    if os.path.exists(ZIP_PATH):
        print("📦 Unzipping dataset... (please wait)")
        !unzip -q -o "{ZIP_PATH}" -d /content/

        # Xử lý cấu trúc thư mục sau khi unzip
        if os.path.exists('/content/camera'):
            !mv /content/camera "{LOCAL_PATH}"
        elif os.path.exists('/content/train') and os.path.exists('/content/test'):
            os.makedirs(LOCAL_PATH, exist_ok=True)
            !mv /content/train "{LOCAL_PATH}/train"
            !mv /content/test "{LOCAL_PATH}/test"
        else:
            print("⚠️ Check ZIP structure!")

        print("✅ Dataset ready at /content/dataset")
    else:
        print("❌ ZIP file not found!")
else:
    print("✅ Dataset already exists locally.")

TRAIN_DIR = os.path.join(LOCAL_PATH, 'train')
TEST_DIR = os.path.join(LOCAL_PATH, 'test')

📦 Unzipping dataset... (please wait)
✅ Dataset ready at /content/dataset


In [5]:
# CELL 5: HYPERPARAMETERS & DATA GENERATORS (ENHANCED)
IMG_SIZE = 48
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.0005
NUM_CLASSES = 7
SEED = 42
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# --- KEY CHANGE: Enhanced Augmentation ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,           # Tăng góc xoay
    width_shift_range=0.2,       # Tăng dịch chuyển
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3], # Thêm độ sáng
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

print("🔄 Loading Data Generators...")
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Class Weights
train_labels = train_generator.classes
class_weights_array = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = dict(enumerate(class_weights_array))
print("⚖️ Class Weights Calculated.")

🔄 Loading Data Generators...
Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
⚖️ Class Weights Calculated.


In [6]:
# CELL 6: BUILD MODEL (Standard CNN)
def build_cnn(input_shape=(48, 48, 1), num_classes=7):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 2
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 4
        layers.Conv2D(512, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(512, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Classification Head
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.4),

        layers.Dense(256, kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.4),

        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = build_cnn()
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)
print("🏗️ Model Built (Standard CNN)")

🏗️ Model Built (Standard CNN)


In [7]:
# CELL 7: TRAINING CALLBACKS
class SaveHistoryCallback(keras.callbacks.Callback):
    def __init__(self, history_path):
        super().__init__()
        self.history_path = history_path
        self.history_data = {'accuracy': [], 'val_accuracy': [], 'loss': [], 'val_loss': [], 'lr': []}

    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            if k in self.history_data:
                self.history_data[k].append(v)
        # Add LR manually if not in logs
        lr = float(self.model.optimizer.learning_rate.numpy())
        if len(self.history_data['lr']) < len(self.history_data['loss']):
             self.history_data['lr'].append(lr)

        with open(self.history_path, 'wb') as f:
            pickle.dump(self.history_data, f)

callbacks = [
    ModelCheckpoint(filepath=BEST_MODEL_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
    SaveHistoryCallback(HISTORY_PATH)
]

In [8]:
# CELL 8: START TRAINING
print("🚀 Starting Training (Method 1)...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

🚀 Starting Training (Method 1)...
Epoch 1/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.1500 - loss: 3.5056
Epoch 1: val_accuracy improved from -inf to 0.01881, saving model to /content/drive/MyDrive/CaptoneProject/checkpoints/method1_aug/best_model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 77s 145ms/step - accuracy: 0.1500 - loss: 3.5052 - val_accuracy: 0.0188 - val_loss: 3.0580 - learning_rate: 5.0000e-04
Epoch 2/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1523 - loss: 3.0584
Epoch 2: val_accuracy improved from 0.01881 to 0.15346, saving model to /content/drive/MyDrive/CaptoneProject/checkpoints/method1_aug/best_model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 32s 89ms/step - accuracy: 0.1523 - loss: 3.0581 - val_accuracy: 0.1535 - val_loss: 2.6934 - learning_rate: 5.0000e-04
Epoch 3/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.1460 - loss: 2.7528
Epoch 3: val_accuracy did not improve from 0.15346
359/359 ━━━━━━━━━━━━━━━━━━━━ 34s 95ms/step - accuracy: 0.14

In [9]:
# CELL 9: EVALUATION
print("\n📊 Evaluating Best Model...")
best_model = keras.models.load_model(BEST_MODEL_PATH)
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"🏆 Test Accuracy: {test_acc:.4f}")

# Classification Report
predictions = best_model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes
print(classification_report(y_true, y_pred, target_names=EMOTIONS))


📊 Evaluating Best Model...
113/113 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.5731 - loss: 1.3405
🏆 Test Accuracy: 0.6293
113/113 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step
              precision    recall  f1-score   support

       angry       0.60      0.50      0.55       958
     disgust       0.40      0.72      0.52       111
        fear       0.49      0.35      0.41      1024
       happy       0.85      0.85      0.85      1774
     neutral       0.54      0.68      0.60      1233
         sad       0.49      0.51      0.50      1247
    surprise       0.75      0.74      0.75       831

    accuracy                           0.63      7178
   macro avg       0.59      0.62      0.60      7178
weighted avg       0.63      0.63      0.63      7178

